In [20]:
import mysql.connector as connect
import random
import smtplib
import os
import getpass

try:
    db = connect.connect(
        host="localhost",
        user="root",
        password=""
    )
    if db.is_connected():
        print("MySQL Database connected successfully...")
        cursor = db.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS bank_system;")
        cursor.execute("USE bank_system;")

except connect.Error as e:
    print(f"Error while connecting to MySQL: {e}")

def send_otp_email(receiver_email):
    otp = random.randint(100000, 999999)
    sender_email = "ritikvishwakarma6595@gmail.com"            
    sender_password = "rcbr ppev ueaz idui"
    subject = "Your Bank Registration OTP"
    body = f"Dear User,\n\nYour OTP is: {otp}\n\n(This is an automated message, please do not reply.)"
    message = f"From: No-Reply Bank <ritikvishwakarma6595@gmail.com>\nTo: {receiver_email}\nSubject: {subject}\n\n{body}"

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, receiver_email, message)
        print(f"OTP sent successfully to {receiver_email}")
        return otp
    except Exception as e:
        print(f"Failed to send OTP: {e}")
        return None

def register():
    username = input("Enter USER: ").strip()
    pin = getpass.getpass("Enter PIN: ").strip()
    receiver_email = input("Enter E-mail: ").strip()

    prefix = "011821000"
    random_part = str(random.randint(4**6, 4**7-1))
    acc = prefix + random_part

    cursor.execute("SELECT * FROM users WHERE username = %s", (username,))
    if cursor.fetchone():
        print("This username already exists!")
        return

    otp = send_otp_email(receiver_email)
    if not otp:
        print("Could not send OTP. Try again later.")
        return

    entered_otp = input("Enter the OTP sent to your email: ").strip()
    if entered_otp != str(otp):
        print("Incorrect OTP. Registration cancelled.")
        return

    try:
        cursor.execute(
            "INSERT INTO users (Account_no, username, email, pin, balance) VALUES (%s, %s, %s, %s, 0)",
            (acc, username, receiver_email, pin)
        )
        db.commit()
        print("Registration successful! You can now log in.")
        print(f"Your A/c number is: {acc}")
    except connect.Error as e:
        print(f"Error while inserting data: {e}")
        db.rollback()


def login():
    username = input("Enter your USER: ").strip()
    pin = getpass.getpass("Enter PIN: ").strip()

    cursor.execute("SELECT * FROM users WHERE username = %s AND pin = %s", (username, pin))
    user = cursor.fetchone()

    if user:
        user_id, Account_no, username, email, pin, balance = user
        print(f"\nLogin Successful! Welcome {username}")
        print(f"Your current balance is: {balance}\n")

        while True:
            print("""
            ====== ATM MENU ======
            1. View Balance
            2. Deposit
            3. Withdraw
            4. View Transaction History
            5. Back
            """)
            choice = input("Enter your choice: ").strip()

            if choice == '1':
                cursor.execute("SELECT balance FROM users WHERE user_id = %s", (user_id,))
                result = cursor.fetchone()
                if result:
                    print(f"Your current balance is: {result[0]}")
                else:
                    print("User not found!")

            elif choice == '2':
                amount = float(input("Enter deposit amount: "))
                cursor.execute("UPDATE users SET balance = balance + %s WHERE user_id = %s", (amount, user_id))
                cursor.execute("INSERT INTO transactions(user_id, trans_type, amount) VALUES(%s, 'deposit', %s)", (user_id, amount))
                db.commit()
                print(f"{amount} deposited successfully!")

            elif choice == '3':
                amount = float(input("Enter withdrawal amount: "))
                cursor.execute("SELECT balance FROM users WHERE user_id = %s", (user_id,))
                bal = cursor.fetchone()[0]
                if amount > bal:
                    print("Insufficient balance!")
                else:
                    cursor.execute("UPDATE users SET balance = balance - %s WHERE user_id = %s", (amount, user_id))
                    cursor.execute("INSERT INTO transactions(user_id, trans_type, amount) VALUES(%s, 'withdraw', %s)", (user_id, amount))
                    db.commit()
                    print(f"{amount} withdrawn successfully!")

            elif choice == '4':
                cursor.execute("SELECT trans_type, amount, trans_date FROM transactions WHERE user_id = %s",(user_id,))
                records = cursor.fetchall()
                if not records:
                    print("No transactions Yet!")
                else:
                    print("Transaction history")
                    for r in records:
                        print(f"{r[2]} | {r[0].upper()} | {r[1]}")
                    

            elif choice == '5':
                print("Thank you for using our ATM! Goodbye!")
                break

            else:
                print("Invalid option. Try again.")

    else:
        print("Invalid username or PIN!")

def admin():
    print("\n===== ADMIN LOGIN =====")
    username = input("Enter Admin USER-NAME: ")
    password = getpass.getpass("Enter 4-digit PIN: ")

    cursor.execute("SELECT * FROM admin WHERE username = %s AND password = %s", (username,password))
    admin = cursor.fetchone()
    if admin:
        print(f"\nWelcome Admin {username}!")
        while True:
            print("""
            1. View All Users
            2. View All Transactions
            3. Delete a User
            4. User Search by A/c no.
            5. Logout
            """)
            choice = input("Enter your choice: ")
            if choice == '1':
                cursor.execute("SELECT user_id, username FROM users")
                users = cursor.fetchall()
                print("User ID", "Username")
                print("-------------------")
                for u in users:
                    print(f"{u[0]} | {u[1]}")

            elif choice == "2":
                cursor.execute("SELECT * FROM transactions")
                trans = cursor.fetchall()
                print("\n--- All Transactions ---")
                for t in trans:
                    print(f"ID: {t[0]} | UserID: {t[1]} | Type: {t[2]} | ₹{t[3]} | Date: {t[4]}")

            elif choice == '3':
                del_id = input("Enter user_id to delete: ")
                cursor.execute("DELETE FROM users WHERE user_id = %s", (del_id,))
                db.commit()
                print("User deleted successfully.")

            elif choice == '4':
                ac = int(input("Enter A/c No: "))
                cursor.execute("SELECT user_id, username, balance, email FROM users WHERE Account_no = %s", (ac,))
                user1 = cursor.fetchone()
                if user1:
                    user_id, username, balance,email = user1
                    print("==============================")
                    print(f"USER_ID: {user_id}")
                    print(f"USERNAME: {username}")
                    print(f"BALANCE: {balance}")
                    print(f"E-MAIL: {email}")
                    print("==============================")
                else:
                    print("No user found with that Account number.")
                    
            elif choice == '5':
                print("Logging out from admin panel...\n")
                break
                


            else:
                print("Invalid option, try again!")
    else:
        print("Invalid admin username or password!")
    
    
def main():
    while True:
        print("""
        ====== PYTHON BANK SYSTEM ======
        1. Register
        2. Login
        3. Admin Login
        4. Exit
        """)
        choice = input("Enter your choice: ").strip()

        if choice == '1':
            register()
        elif choice == '2':
            login()
        elif choice == '3':
            admin()
        elif choice == '4':
            print("Thank you for banking with us!")
            break
        else:
            print("Invalid choice. Try again.")

main()


MySQL Database connected successfully...

        ====== PYTHON BANK SYSTEM ======
        1. Register
        2. Login
        3. Admin Login
        4. Exit
        


Enter your choice:  1
Enter USER:  ritik
Enter PIN:  ········
Enter E-mail:  ritikvishwakarma07@gmail.com


OTP sent successfully to ritikvishwakarma07@gmail.com


Enter the OTP sent to your email:  650421


Registration successful! You can now log in.
Your A/c number is: 0118210005326

        ====== PYTHON BANK SYSTEM ======
        1. Register
        2. Login
        3. Admin Login
        4. Exit
        


...

USER_ID: 10
USERNAME: ritik
BALANCE: 0
E-MAIL: ritikvishwakarma07@gmail.com

            1. View All Users
            2. View All Transactions
            3. Delete a User
            4. User Search by A/c no.
            5. Logout
            


Enter your choice:  5


Logging out from admin panel...


        ====== PYTHON BANK SYSTEM ======
        1. Register
        2. Login
        3. Admin Login
        4. Exit
        


Enter your choice:  4


Thank you for banking with us!


In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
